In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium==4.26.1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,502 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [ ]:
# Import modules
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import drive
import os
import json

In [ ]:
# Install the correct version of chromedriver
chromedriver_autoinstaller.install()

In [ ]:
# Set up the Chrome driver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome(options=options)

In [ ]:
# JSON 파일 로드
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/trump_youtube_url_title_date.json', 'r') as f:
    video_data = json.load(f)

In [ ]:
# 결과를 저장할 리스트와 파일 경로 설정
output_data = []
output_file_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/comments_data.jsonl'
processed_videos_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/processed_videos.json'

In [ ]:
# 이미 처리된 비디오 URL 로드
if os.path.exists(processed_videos_path):
    with open(processed_videos_path, 'r', encoding='utf-8') as f:
        processed_videos = set(json.load(f))
else:
    processed_videos = set()

In [ ]:
# 각 비디오에 대해 댓글 스크래핑
for idx, video in enumerate(video_data):
    url = video['url']
    video_title = video['title']
    video_date = video['date']

    if url in processed_videos:
        print(f"Skipping already processed video {idx+1}/{len(video_data)}: {video_title}")
        continue

    try:
        print(f"Processing video {idx+1}/{len(video_data)}: {video_title}")

        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 댓글 로드를 위한 스크롤 다운
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # 모든 댓글을 로드하기 위한 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scroll_attempts = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= 3:
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        # 댓글 요소 찾기
        comments = driver.find_elements(By.CSS_SELECTOR, 'ytd-comment-thread-renderer')
        print(f"Found {len(comments)} comments")

        for comment in comments:
            try:
                # 댓글 텍스트 추출
                comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text').text.strip()

                # 댓글 작성 날짜 추출
                date_element = comment.find_element(By.CSS_SELECTOR, '#published-time-text')
                date = date_element.text.strip()

                # 좋아요 수 추출 (파싱 없이 원본 그대로 저장)
                likes_element = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle')
                likes_text = likes_element.text.strip()

                # 데이터 저장
                output_data.append({
                    'video_url': url,
                    'video_title': video_title,
                    'video_date': video_date,
                    'comment_text': comment_text,
                    'comment_date': date,
                    'comment_likes': likes_text  # 원본 텍스트 그대로 저장
                })
            except Exception as e:
                print(f"Error processing a comment: {e}")
                continue

        # 중간 결과 저장
        with open(output_file_path, 'a', encoding='utf-8') as f_out:
            for data in output_data:
                json_record = json.dumps(data, ensure_ascii=False)
                f_out.write(json_record + '\n')
            output_data = []  # 리스트 초기화

        # 처리된 비디오 URL 저장
        processed_videos.add(url)
        with open(processed_videos_path, 'w', encoding='utf-8') as f_processed:
            json.dump(list(processed_videos), f_processed, ensure_ascii=False)

        print(f"Finished processing video {idx+1}/{len(video_data)}")
    except Exception as e:
        print(f"Error processing video {url}: {e}")
        continue

Processing video 1/113: Together.
Found 1113 comments
Finished processing video 1/113
Processing video 2/113: Tomorrow.
Found 1111 comments
Finished processing video 2/113
Processing video 3/113: A Special Message from President Donald J. Trump
Found 1088 comments
Finished processing video 3/113
Processing video 4/113: We Fight
Found 870 comments
Finished processing video 4/113
Processing video 5/113: Trump Force 47 - Weekly Recap
Found 773 comments
Finished processing video 5/113
Processing video 6/113: MAGA Women Respond to Kamala Surrogate Mark Cuban
Found 708 comments
Finished processing video 6/113
Processing video 7/113: I'm Not With Her
Found 866 comments
Finished processing video 7/113
Processing video 8/113: President Trump Tells the Hilarious Story of How the Garbage Truck and the Orange Vest Came To Be
Found 760 comments
Finished processing video 8/113
Processing video 9/113: Kamala and Joe call us all garbage. I call you the heart and soul of America.
Found 739 comments
Fin

KeyboardInterrupt: 

In [ ]:
# 이미 처리된 비디오 URL 로드
if os.path.exists(processed_videos_path):
    with open(processed_videos_path, 'r', encoding='utf-8') as f:
        processed_videos = set(json.load(f))
else:
    processed_videos = set()

In [ ]:
# 각 비디오에 대해 댓글 스크래핑
for idx, video in enumerate(video_data):
    url = video['url']
    video_title = video['title']
    video_date = video['date']

    if url in processed_videos:
        print(f"Skipping already processed video {idx+1}/{len(video_data)}: {video_title}")
        continue

    try:
        print(f"Processing video {idx+1}/{len(video_data)}: {video_title}")

        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 댓글 로드를 위한 스크롤 다운
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # 모든 댓글을 로드하기 위한 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scroll_attempts = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= 3:
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        # 댓글 요소 찾기
        comments = driver.find_elements(By.CSS_SELECTOR, 'ytd-comment-thread-renderer')
        print(f"Found {len(comments)} comments")

        for comment in comments:
            try:
                # 댓글 텍스트 추출
                comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text').text.strip()

                # 댓글 작성 날짜 추출
                date_element = comment.find_element(By.CSS_SELECTOR, '#published-time-text')
                date = date_element.text.strip()

                # 좋아요 수 추출 (파싱 없이 원본 그대로 저장)
                likes_element = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle')
                likes_text = likes_element.text.strip()

                # 데이터 저장
                output_data.append({
                    'video_url': url,
                    'video_title': video_title,
                    'video_date': video_date,
                    'comment_text': comment_text,
                    'comment_date': date,
                    'comment_likes': likes_text  # 원본 텍스트 그대로 저장
                })
            except Exception as e:
                print(f"Error processing a comment: {e}")
                continue

        # 중간 결과 저장
        with open(output_file_path, 'a', encoding='utf-8') as f_out:
            for data in output_data:
                json_record = json.dumps(data, ensure_ascii=False)
                f_out.write(json_record + '\n')
            output_data = []  # 리스트 초기화

        # 처리된 비디오 URL 저장
        processed_videos.add(url)
        with open(processed_videos_path, 'w', encoding='utf-8') as f_processed:
            json.dump(list(processed_videos), f_processed, ensure_ascii=False)

        print(f"Finished processing video {idx+1}/{len(video_data)}")
    except Exception as e:
        print(f"Error processing video {url}: {e}")
        continue

Skipping already processed video 1/113: Together.
Skipping already processed video 2/113: Tomorrow.
Skipping already processed video 3/113: A Special Message from President Donald J. Trump
Skipping already processed video 4/113: We Fight
Skipping already processed video 5/113: Trump Force 47 - Weekly Recap
Skipping already processed video 6/113: MAGA Women Respond to Kamala Surrogate Mark Cuban
Skipping already processed video 7/113: I'm Not With Her
Skipping already processed video 8/113: President Trump Tells the Hilarious Story of How the Garbage Truck and the Orange Vest Came To Be
Skipping already processed video 9/113: Kamala and Joe call us all garbage. I call you the heart and soul of America.
Skipping already processed video 10/113: "How Do You Like My Garbage Truck? This Truck is in Honor of Kamala and Joe Biden."
Skipping already processed video 11/113: Hall of Fame Quarterback Brett Favre Speaks at President Trump's Rally in Green Bay, Wisconsin
Skipping already processed v

In [ ]:
# 이미 처리된 비디오 URL 로드
if os.path.exists(processed_videos_path):
    with open(processed_videos_path, 'r', encoding='utf-8') as f:
        processed_videos = set(json.load(f))
else:
    processed_videos = set()

In [ ]:
# 각 비디오에 대해 댓글 스크래핑
for idx, video in enumerate(video_data):
    url = video['url']
    video_title = video['title']
    video_date = video['date']

    if url in processed_videos:
        print(f"Skipping already processed video {idx+1}/{len(video_data)}: {video_title}")
        continue

    try:
        print(f"Processing video {idx+1}/{len(video_data)}: {video_title}")

        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 댓글 로드를 위한 스크롤 다운
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # 모든 댓글을 로드하기 위한 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scroll_attempts = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= 3:
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        # 댓글 요소 찾기
        comments = driver.find_elements(By.CSS_SELECTOR, 'ytd-comment-thread-renderer')
        print(f"Found {len(comments)} comments")

        for comment in comments:
            try:
                # 댓글 텍스트 추출
                comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text').text.strip()

                # 댓글 작성 날짜 추출
                date_element = comment.find_element(By.CSS_SELECTOR, '#published-time-text')
                date = date_element.text.strip()

                # 좋아요 수 추출 (파싱 없이 원본 그대로 저장)
                likes_element = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle')
                likes_text = likes_element.text.strip()

                # 데이터 저장
                output_data.append({
                    'video_url': url,
                    'video_title': video_title,
                    'video_date': video_date,
                    'comment_text': comment_text,
                    'comment_date': date,
                    'comment_likes': likes_text  # 원본 텍스트 그대로 저장
                })
            except Exception as e:
                print(f"Error processing a comment: {e}")
                continue

        # 중간 결과 저장
        with open(output_file_path, 'a', encoding='utf-8') as f_out:
            for data in output_data:
                json_record = json.dumps(data, ensure_ascii=False)
                f_out.write(json_record + '\n')
            output_data = []  # 리스트 초기화

        # 처리된 비디오 URL 저장
        processed_videos.add(url)
        with open(processed_videos_path, 'w', encoding='utf-8') as f_processed:
            json.dump(list(processed_videos), f_processed, ensure_ascii=False)

        print(f"Finished processing video {idx+1}/{len(video_data)}")
    except Exception as e:
        print(f"Error processing video {url}: {e}")
        continue

Skipping already processed video 1/113: Together.
Skipping already processed video 2/113: Tomorrow.
Skipping already processed video 3/113: A Special Message from President Donald J. Trump
Skipping already processed video 4/113: We Fight
Skipping already processed video 5/113: Trump Force 47 - Weekly Recap
Skipping already processed video 6/113: MAGA Women Respond to Kamala Surrogate Mark Cuban
Skipping already processed video 7/113: I'm Not With Her
Skipping already processed video 8/113: President Trump Tells the Hilarious Story of How the Garbage Truck and the Orange Vest Came To Be
Skipping already processed video 9/113: Kamala and Joe call us all garbage. I call you the heart and soul of America.
Skipping already processed video 10/113: "How Do You Like My Garbage Truck? This Truck is in Honor of Kamala and Joe Biden."
Skipping already processed video 11/113: Hall of Fame Quarterback Brett Favre Speaks at President Trump's Rally in Green Bay, Wisconsin
Skipping already processed v

미국 대선 (11월 5일 이후의 댓글들은 필터링)

In [3]:
import pandas as pd

file_path = "/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/trump_youtube_comment_commentdate_likes.jsonl"

df = pd.read_json(file_path, lines=True)

In [4]:
len(df)

83525

In [5]:
print(df.head(5))

                                     video_url video_title   video_date  \
0  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
1  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
2  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
3  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
4  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   

                                        comment_text comment_date  \
0     This isn’t a campaign ad. This is a victory ad  13 days ago   
1  As a former Democrat myself, its awesome to se...  13 days ago   
2                    The Avengers trailer we needed.  2 weeks ago   
3  Just an insanely good ad.  Really shows how mu...   4 days ago   
4       Beautiful, Hope you win to save our world !!  2 weeks ago   

  comment_likes  
0          4.8K  
1          1.3K  
2          4.4K  
3            70  
4           126  


In [6]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

['13 days ago' '2 weeks ago' '4 days ago' '1 day ago' '11 days ago'
 '6 days ago' '12 days ago' '2 days ago' '9 days ago'
 '2 weeks ago (edited)' '19 hours ago' '8 days ago' '3 days ago'
 '7 days ago (edited)' '5 days ago' '7 days ago' '10 days ago'
 '10 days ago (edited)' '5 days ago (edited)' '21 hours ago (edited)'
 '8 days ago (edited)' '3 days ago (edited)' '2 days ago (edited)'
 '4 days ago (edited)' '7 hours ago' '1 day ago (edited)' '18 hours ago'
 '6 days ago (edited)' '10 hours ago (edited)' '17 hours ago'
 '5 hours ago' '4 hours ago' '6 hours ago' '11 minutes ago' '1 hour ago'
 '16 hours ago' '15 hours ago (edited)' '10 hours ago' '2 hours ago'
 '15 hours ago' '3 hours ago' '20 hours ago' '14 hours ago' '9 hours ago'
 '12 days ago (edited)' '11 days ago (edited)' '13 days ago (edited)'
 '9 days ago (edited)' '3 weeks ago' '3 weeks ago (edited)' '21 hours ago'
 '23 hours ago' '4 weeks ago' '4 weeks ago (edited)' '1 month ago'
 '11 hours ago' '1 month ago (edited)' '23 hours a

In [7]:
# video_dates 컬럼의 고유한 값 확인
unique_video_dates = df['video_date'].unique()

# 고유한 값 출력
print(unique_video_dates) #

['Nov 4, 2024' 'Nov 3, 2024' 'Nov 2, 2024' 'Nov 1, 2024' 'Oct 31, 2024'
 'Oct 30, 2024' 'Oct 28, 2024' 'Oct 27, 2024' 'Oct 26, 2024'
 'Oct 25, 2024' 'Oct 24, 2024' 'Oct 23, 2024' 'Oct 22, 2024'
 'Oct 20, 2024' 'Oct 19, 2024' 'Oct 18, 2024' 'Oct 17, 2024'
 'Oct 16, 2024' 'Oct 15, 2024' 'Oct 14, 2024' 'Oct 13, 2024'
 'Oct 12, 2024' 'Oct 11, 2024' 'Oct 9, 2024' 'Oct 8, 2024' 'Oct 5, 2024'
 'Oct 4, 2024' 'Oct 3, 2024' 'Oct 2, 2024' 'Sep 30, 2024' 'Sep 27, 2024'
 'Sep 26, 2024' 'Sep 24, 2024' 'Sep 23, 2024' 'Sep 22, 2024'
 'Sep 19, 2024' 'Sep 16, 2024' 'Sep 15, 2024' 'Sep 14, 2024'
 'Sep 13, 2024' 'Sep 10, 2024' 'Sep 6, 2024' 'Sep 4, 2024' 'Sep 3, 2024'
 'Aug 28, 2024' 'Aug 26, 2024' 'Aug 23, 2024' 'Aug 22, 2024'
 'Aug 21, 2024' 'Aug 20, 2024' 'Aug 16, 2024' 'Aug 14, 2024'
 'Aug 13, 2024' 'Aug 10, 2024' 'Aug 7, 2024' 'Aug 6, 2024' 'Aug 5, 2024'
 'Jul 30, 2024' 'Jul 26, 2024' 'Jul 25, 2024' 'Jul 21, 2024']


In [8]:
# 현재 2024년 11월 25일 기준 (크롤링은 23일에 진행)

In [11]:
from datetime import datetime, timedelta
import re

In [12]:
def parse_comment_date(comment_date):
    try:
        # "(edited)" 제거
        comment_date = re.sub(r"\(edited\)", "", comment_date).strip()

        # "days ago" 형식 처리
        if "day" in comment_date:
            days_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(days=days_ago)

        # "week" 형식 처리
        elif "week" in comment_date:
            weeks_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(weeks=weeks_ago)

        # "month" 형식 처리
        elif "month" in comment_date:
            months_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(days=months_ago * 30)  # 대략 30일 기준

        # "hour" 형식 처리
        elif "hour" in comment_date:
            hours_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(hours=hours_ago)

        # "minute" 형식 처리
        elif "minute" in comment_date:
            minutes_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(minutes=minutes_ago)

        # 기본적으로 None 반환 (기타 형식에 대해 처리할 수 없을 경우)
        else:
            return None

        # 날짜만 반환 (시간과 마이크로초 제거)
        return result_date.date()

    except Exception as e:
        print(f"Error parsing comment_date '{comment_date}': {e}")
        return None


In [13]:
# 'comment_date'를 datetime으로 변환하여 새로운 컬럼 생성
df['parsed_comment_date'] = df['comment_date'].apply(parse_comment_date)

In [14]:
# 결과 출력 (테스트용)
print(df[['comment_date', 'parsed_comment_date']].head(10))

  comment_date parsed_comment_date
0  13 days ago          2024-11-11
1  13 days ago          2024-11-11
2  2 weeks ago          2024-11-10
3   4 days ago          2024-11-20
4  2 weeks ago          2024-11-10
5  2 weeks ago          2024-11-10
6  2 weeks ago          2024-11-10
7    1 day ago          2024-11-23
8  2 weeks ago          2024-11-10
9  2 weeks ago          2024-11-10


In [15]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['parsed_comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

# 마지막 비디오 영상이 Jul 21, 2024이며, 크롤링한 2024년 11월 23일 기준 4 month ago 보다 이전의 댓글 날짜는 없다.
# 즉 7월 21일 ~ 7월 27일 -> 7월 27일로 통일된다. (마찬가지로 2024, 7, 28 ~ 2024, 8, 26는 2024, 8, 26로 통일됨.)
# 어차피 댓글 날짜를 추출한 이유는 필터링이 목적이므로 크게 상관없다.

[datetime.date(2024, 11, 11) datetime.date(2024, 11, 10)
 datetime.date(2024, 11, 20) datetime.date(2024, 11, 23)
 datetime.date(2024, 11, 13) datetime.date(2024, 11, 18)
 datetime.date(2024, 11, 12) datetime.date(2024, 11, 22)
 datetime.date(2024, 11, 15) datetime.date(2024, 11, 16)
 datetime.date(2024, 11, 21) datetime.date(2024, 11, 17)
 datetime.date(2024, 11, 19) datetime.date(2024, 11, 14)
 datetime.date(2024, 11, 24) datetime.date(2024, 11, 3)
 datetime.date(2024, 10, 27) datetime.date(2024, 10, 25)
 datetime.date(2024, 9, 25) datetime.date(2024, 8, 26)
 datetime.date(2024, 7, 27)]


In [16]:
print(df.head(5))

                                     video_url video_title   video_date  \
0  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
1  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
2  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
3  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
4  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   

                                        comment_text comment_date  \
0     This isn’t a campaign ad. This is a victory ad  13 days ago   
1  As a former Democrat myself, its awesome to se...  13 days ago   
2                    The Avengers trailer we needed.  2 weeks ago   
3  Just an insanely good ad.  Really shows how mu...   4 days ago   
4       Beautiful, Hope you win to save our world !!  2 weeks ago   

  comment_likes parsed_comment_date  
0          4.8K          2024-11-11  
1          1.3K          2024-11-11  
2          4.4K     

In [17]:
# 'comment_date' 컬럼 삭제
df = df.drop(columns=['comment_date'])

In [18]:
print(df.head(5))

                                     video_url video_title   video_date  \
0  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
1  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
2  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
3  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   
4  https://www.youtube.com/watch?v=knkdSsAlDhc   Together.  Nov 4, 2024   

                                        comment_text comment_likes  \
0     This isn’t a campaign ad. This is a victory ad          4.8K   
1  As a former Democrat myself, its awesome to se...          1.3K   
2                    The Avengers trailer we needed.          4.4K   
3  Just an insanely good ad.  Really shows how mu...            70   
4       Beautiful, Hope you win to save our world !!           126   

  parsed_comment_date  
0          2024-11-11  
1          2024-11-11  
2          2024-11-10  
3          2024-11-20  
4       

In [19]:
len(df)

83525

In [20]:
# 날짜 기준 설정 (2024-11-04 이후는 삭제)
date_threshold = datetime(2024, 11, 4).date()

In [21]:
# 'parsed_comment_date'를 datetime 형식으로 변환하여 날짜만 비교
df['parsed_comment_date'] = pd.to_datetime(df['parsed_comment_date']).dt.date

In [22]:
# 날짜 조건에 맞는 행만 남기기 (2024-11-04 이후의 데이터 삭제)
df = df[df['parsed_comment_date'] <= date_threshold]

In [23]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['parsed_comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

[datetime.date(2024, 11, 3) datetime.date(2024, 10, 27)
 datetime.date(2024, 10, 25) datetime.date(2024, 9, 25)
 datetime.date(2024, 8, 26) datetime.date(2024, 7, 27)]


In [24]:
len(df)

74809

In [25]:
print(df.head(5))

                                        video_url  \
4263  https://www.youtube.com/watch?v=d608OHI_muM   
4334  https://www.youtube.com/watch?v=d608OHI_muM   
4366  https://www.youtube.com/watch?v=d608OHI_muM   
4369  https://www.youtube.com/watch?v=d608OHI_muM   
4371  https://www.youtube.com/watch?v=d608OHI_muM   

                        video_title   video_date  \
4263  Trump Force 47 - Weekly Recap  Nov 1, 2024   
4334  Trump Force 47 - Weekly Recap  Nov 1, 2024   
4366  Trump Force 47 - Weekly Recap  Nov 1, 2024   
4369  Trump Force 47 - Weekly Recap  Nov 1, 2024   
4371  Trump Force 47 - Weekly Recap  Nov 1, 2024   

                                           comment_text comment_likes  \
4263  as a Turkish guy, please save your culture and...           249   
4334                       Love you president Trump 🇺🇸🫡            18   
4366  Fight, Fight and Fight!!! Donald John Trump Fo...            21   
4369  Truckers For Trump!! \nToo Big to Rig!!\nTrump...            48   
437

In [26]:
output_file_path = "/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/trump_youtube_comment_filtered.jsonl"
df.to_json(output_file_path, orient='records', lines=True)